In [1]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/ColabNotebooks/Data/MTech_Data/Exp6_Myo5_20200221/"
#fingers = ['1T', '2I', '3M', '4R', '5L']

Mounted at /content/drive


In [2]:
import pandas as pd
path = '/content/drive/MyDrive/ColabNotebooks/Data/MTech_Data/Exp6_Myo5_20200221/'

#!ls '/content/drive/MyDrive/ColabNotebooks/Data/MTech_Data/Exp10_5KeyPress_MultipleTimes_20200228/all/'
fingers = ['1_fingerspread','2_doubletap','3_wavein', '4_waveout','5_fist']
#fingers = ['1T','2I','3M', '4R', '5L']

In [3]:
import glob
import sys
import pandas
minlen = sys.maxsize
#print(minlen)
for exp in fingers :
    files = glob.glob(path + exp + '/emg*.csv')
    #print(files)    
    for file in files:
        emgData = pandas.read_csv(file)
        namepos = file.index('-')
        name = file[namepos:]
        if (minlen > len(emgData[emgData.columns[0]])):
          minlen = len(emgData[emgData.columns[0]])
print(minlen)

1996


In [ ]:
factor227 = (int)(minlen/227)
factoredminlen = factor227*227
factoredminlen

1816

In [ ]:
############################## Read Multichannel Data
import pandas
import datetime
import os
import math
import glob
import sys
import  numpy as np
from scipy import signal
#from matplotlib.mlab import PCA as mlabPCA
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy 
from statsmodels import robust
from sklearn.metrics import accuracy_score 
%matplotlib inline
#colnames = ['emg1', 'emg2', 'emg3', 'emg4', 'emg5', 'emg6', 'emg7', 'emg8']
colnames = ['emg8']
print(colnames)
inputShape = []
complete_emg =[]
X=[]
index =0 
count =0
Y=[]
channelsize = 40
minlen = sys.maxsize
for exp in fingers :
    files = glob.glob(path + exp + '/emg*.csv')
    count =0
    #print(files)    
    for file in files:
        count = count + 1
        #print(file)
        emgData = pandas.read_csv(file)
        namepos = file.index('-')
        name = file[namepos:]
        complete_emg = []        
        for data in emgData.columns:
          #if data == 'emg1' or data == 'emg7' or data == 'emg8':
          #if 'timestamp' in data:
          if not data == 'emg8':
            continue
          else : 
            currentemg = data[3]
            index += 1
            complete_emg.append(emgData[data][0:factoredminlen])
        Y.append(int(exp[0])-1)
        X.append(complete_emg)

print(len(X))
print(len(X[0]))
print(len(X[0][0]))
print("Current Emg {}".format(currentemg))
############################## Read Multichannel Data

['emg8']
150
1
1816
Current Emg 8


In [ ]:
import torch
import numpy
rawX = torch.tensor(np.stack(X))
print(rawX.dtype)
rawX = rawX.type(torch.FloatTensor)
print(rawX.dtype)
rawY = torch.from_numpy(numpy.array(Y, dtype='long'))
print(str(rawX.shape) + "," + str(rawY.shape))

torch.int64
torch.float32
torch.Size([150, 1, 1816]),torch.Size([150])


In [ ]:
#Create Short Frames
import numpy
newX = []
newY = []
for csX, csY in zip(rawX,rawY):
  x = np.reshape(csX, (-1, 227))
  y=[]
  dupliclen = x.shape[0]
  y = [csY for i in range(dupliclen)] 
  if (len(newX) == 0):
    newX = x
    newY = y
  else :
    newX = numpy.append(newX, x, axis = 0)
    newY =numpy.append(newY, y)
print(len(newX))
print(len(newX[0]))
print(newX.shape)
print(len(newY))
X = newX
Y = newY
#Create Short Frames

1200
227
(1200, 227)
1200


In [ ]:
#Save short Frames To File
import numpy as np
mergedInput = np.column_stack((X,Y))
print(len(mergedInput))
ffname = "EmgElectrode8_Frame"+str(factoredminlen)+"to227.csv"
framewisefilename = path + ffname
numpy.savetxt(framewisefilename, mergedInput, delimiter=',')
print(framewisefilename)

1200
/content/drive/MyDrive/ColabNotebooks/Data/MTech_Data/Exp6_Myo5_20200221/EmgElectrode8_Frame1816to227.csv


In [ ]:
############################################### Add features to new row############################################### 
import pandas
import datetime
import os
import math
import glob
import  numpy as np
from scipy import signal
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy 
from statsmodels import robust
from sklearn.metrics import accuracy_score 
%matplotlib inline
# Standard Error - the standard error of the mean (or standard error of measurement) of the values in the  array
# Mean Absolute Deviation - interval amoung each data value of a dataset from its means. find variation in data set

#input_cols = ['Mean Avg', 'Variance', 'Standard Deviation','Skewness', 'Kurtosis', 'Standar Error',
#              'Mean Absolute Deviation', 'Root Mean Square', 'WaveLength', 'Mean Frequency','Median Frequency',
#              'MeanAvg_Acc_x', 'MeanAvg_Acc_y', 'MeanAvg_Acc_z', 'MeanAvg_Gyro_x', 'MeanAvg_Gyro_y', 'MeanAvg_Gyro_z',
#              'Orien_x', 'Orien_y', 'Orien_z', 'Orien_w','Orien_Euler_roll', 'Orien_Euler_pitch','Orien_Euler_yaw', 'Result']

#result = []
#result.append([])
input_cols = ['Mean Avg', 'Variance', 'Standard Deviation','Skewness', 'Kurtosis', 'Standar Error',
              'Mean Absolute Deviation', 'Root Mean Square', 'WaveLength', 'Mean Frequency','Median Frequency']
#fingers = ['1T', '2I', '3M', '4R', '5L']
#fingers= ['1_fingerspread','2_doubletap','3_wavein', '4_waveout','5_fist']

minlen = factoredminlen
sample_size = 227
repeat = 1 #(int)(minlen/sample_size)+1
result = []
result.append([])
for i in range(0,repeat, 1):
  for val in input_cols:
        result[0].append(val)    

result[0].append('Result')
input_cols = result[0]
#Data Experiment 12 sec - 2 Electrode data in One Figure
index =0 
count =0
#max_count = 60
resfilename =  path + "2_Features_" +str(sample_size) + "_EMG8_5KeyPress.csv"
temp =0

for exp in fingers :
    files = glob.glob(path + exp + '/emg*.csv')
    count =0
    #print(files)
    for file in files:
        count = count + 1
        emgData = pandas.read_csv(file)
        namepos = file.index('-')
        name = file[namepos:]
        #print(emgData.shape)
        for data in emgData.columns:
            list = ['emg1','emg7','emg8']
            #if data not in list:
            #    continue
            if not 'emg8' in data:
                continue

            complete_emg = emgData[data]
            
            for i in range(0, factoredminlen, sample_size):
                #if i >= 1135:
                #  continue
                result.append([]) 
                index += 1
                emg = complete_emg[i:i+sample_size]
                emg = emg.dropna()
                emg -= np.mean(emg)
                
                ns = emg.size
                fs = ns
                T = 1/fs
                
                fist_inst = emg
                len_col = len(fist_inst)
                
                if len_col is 0:
                    print("continue length {0}={1}".format(t, colname))
                    continue
                    
                fist_inst = pandas.to_numeric(fist_inst)
                
                #MAV sum(x)/N
                mean = np.mean(fist_inst)
                result[index].append(mean)
                
                #Variance = sum(x n - mean)**2/ n-1
                var = np.var(fist_inst)
                result[index].append(var)
                
                #Standard Deviation
                std = np.std(fist_inst)
                result[index].append(std)
                
                #Skewness - is by taking the mean of the cubes of differences of each point from the mean
                #and then dividing it by the cube of the standard deviation. 
                
                skew = scipy.stats.skew(fist_inst, axis=0)
                result[index].append(skew)
                
                #Kurtosis
                kurt =scipy.stats.kurtosis(fist_inst,axis=0,fisher=False) 
                result[index].append(kurt)
                
                #standard error 
                sem = scipy.stats.sem(fist_inst)
                result[index].append(sem)
                
                #mean absolute deviation
                mad = robust.mad(fist_inst)
                result[index].append(mad)
                
                #RMS sqrt(1/N . sum (x ^2))
                squareandsum = np.square(fist_inst)
                temp = np.sum(squareandsum)/len_col
                result[index].append(math.sqrt(temp))
                
                #WL sum 
                diff_fist = np.diff(fist_inst[0:len_col])
                diff_fist = np.absolute(diff_fist)    
                sumvalue = np.sum(diff_fist)    
                
                #print(sumvalue)
                result[index].append(sumvalue)
                
                #*************************Frequency Domain***********************
                len_colf=len_col
                #print("{0}={1}={2}".format(x, len_colf, len_col))
                psd = np.abs(np.fft.fft(fist_inst, n=256))**2
                #plt.show((np.fft.fft(fist_inst, n=len_colf))[0])
                #if index ==1: 
                  #print(fist_inst.values)
                  #tempdata = fist_inst.values
                  #plt.show(tempdata)
                  #plt.plot(tempdata.index, tempdata.values)
                  #plt.show(np.abs(np.fft.fft(fist_inst, n=256)))
                  #plt.show(10*math.log(psd))
                  #print(np.fft.fft(fist_inst, n=len_colf))
                  #print(psd)
                #break
                #Mean frequency sum(P)/M      
                sumfreq = np.sum(psd)    
                result[index].append(sumfreq/len_colf)
                
                #Median Frequency SUM(P)/2  
                result[index].append(sumfreq/2)
                
                #Append Result
                result[index].append(int(exp[0])-1)
            continue
            
print('Index = {0}'.format(index))  
print("Writing to file = {0}".format(resfilename))
np.where(result=='[', '', result)
np.where(result==']', '', result)
np.savetxt(str(resfilename), result, delimiter=",", fmt='%s')

Index = 1200
Writing to file = /content/drive/MyDrive/ColabNotebooks/Data/MTech_Data/Exp6_Myo5_20200221/2_Features_227_EMG8_5KeyPress.csv


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import accuracy_score 
%matplotlib inline

input_cols_all = ['Mean Avg', 'Variance', 'Standard Deviation','Skewness', 'Kurtosis', 'Standar Error', 
                  'Mean Absolute Deviation', 'Root Mean Square', 'WaveLength', 'Mean Frequency','Median Frequency', 'Result']
##### make LDA components according to file n_components=5/6
data = pd.read_csv(resfilename, usecols=input_cols_all)

input_cols = input_cols_all[0:-1]
print(input_cols)
print (data[1:5])


['Mean Avg', 'Variance', 'Standard Deviation', 'Skewness', 'Kurtosis', 'Standar Error', 'Mean Absolute Deviation', 'Root Mean Square', 'WaveLength', 'Mean Frequency', 'Median Frequency']
       Mean Avg    Variance  Standard Deviation  Skewness  Kurtosis  \
1  9.077418e-16  619.196608           24.883661  0.591317  6.249387   
2  1.189455e-15  579.176968           24.066096  0.164076  4.142716   
3 -3.756173e-16  520.378001           22.811795 -0.143469  5.091893   
4 -1.525945e-16  429.373673           20.721334  0.498691  4.291744   

   Standar Error  Mean Absolute Deviation  Root Mean Square  WaveLength  \
1       1.655237                16.308624         24.883661      5993.0   
2       1.600853                19.273829         24.066096      6145.0   
3       1.517418                10.378216         22.811795      5377.0   
4       1.378363                14.826022         20.721334      5407.0   

   Mean Frequency  Median Frequency  Result  
1   158514.331580      1.799138e+07

In [ ]:
# LDA with scaling 
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import train_test_split

X = data.iloc[:,:-1].copy()
y = data['Result'].copy()
#X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, shuffle=True) 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  
Xtest_copy = X_test
print('X_train={}, X_test={}'.format(X_train.shape, X_test.shape))
sc = StandardScaler()  
X_train_sc = sc.fit_transform(X_train)  
X_test = sc.transform(X_test) 
lda = LDA(n_components=4)  
X_train_lda = lda.fit_transform(X_train_sc, y_train)  
X_test = lda.transform(X_test)  

from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(max_depth=100, random_state=0)
classifier.fit(X_train_lda, y_train)  
y_pred = classifier.predict(X_test)  

y_result = classifier.predict(X_train_lda)  
print('Accuracy' + str(accuracy_score(y_train, y_result))) 

from sklearn.metrics import confusion_matrix  
from sklearn.metrics import accuracy_score

cm = confusion_matrix(y_test, y_pred)  
print(cm)  
print('Accuracy' + str(accuracy_score(y_test, y_pred))) 

X_train=(960, 11), X_test=(240, 11)
Accuracy1.0
[[31  0  3  2  4]
 [ 0 41  0  0  0]
 [ 9  1 43  4  0]
 [ 8  0  4 32  0]
 [ 0  0  0  0 58]]
Accuracy0.8541666666666666


In [ ]:
len(Xtest_copy)
Xtest_copy

,Mean Avg,Variance,Standard Deviation,Skewness,Kurtosis,Standar Error,Mean Absolute Deviation,Root Mean Square,WaveLength,Mean Frequency,Median Frequency
198,-3.130144e-16,390.008267,19.748627,-0.167192,5.211371,1.313659,10.378216,19.748627,4929.0,99842.116400,1.133208e+07
328,2.112847e-16,44.326108,6.657785,0.524384,6.885378,0.442869,4.447807,6.657785,1758.0,11347.483553,1.287939e+06
45,3.130144e-16,240.073667,15.494311,-0.242980,6.199339,1.030666,8.895613,15.494311,3568.0,61458.858817,6.975580e+06
263,0.000000e+00,11.212211,3.348464,0.600772,6.236959,0.222737,2.965204,3.348464,849.0,2870.325914,3.257820e+05
621,1.565072e-16,266.539851,16.326048,-0.357406,10.350211,1.085993,7.413011,16.326048,3675.0,68234.201945,7.744582e+06
...,...,...,...,...,...,...,...,...,...,...,...
1127,5.477752e-16,485.875138,22.042576,0.074687,5.745581,1.466250,16.308624,22.042576,5881.0,124384.035398,1.411759e+07
55,-3.130144e-17,278.291486,16.682071,0.114924,4.215997,1.109675,11.860818,16.682071,4144.0,71242.620505,8.086037e+06
500,-2.073721e-16,307.067748,17.523349,0.210825,9.099369,1.165636,7.413011,17.523349,3607.0,78609.343554,8.922160e+06
687,3.912680e-17,170.284151,13.049297,-0.408405,8.739725,0.868026,5.930409,13.049297,2862.0,43592.742572,4.947776e+06


In [ ]:
mapX =[]
mapY = []
index =0
for i, j in zip(y_pred, y_test):
  if i == j :
    mapX.append(Xtest_copy)
    mapY.append(y_pred)
print(len(mapX))

205


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

#print(Xtest_copy.shape)
pdX = pd.DataFrame(Xtest_copy[0])
(pdX.shape)
# print(len(pdX))
# corr = pdX.corr()
# print(corr)

KeyError: ignored

In [ ]:
ax1 = sns.heatmap(
    Xtest_copy.T, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
)